In [ ]:
import torch

torch.cuda.is_available()

## Load monkey model

In [ ]:
# from laminr.neuron_models import monkey_v1

In [ ]:
# response_predicting_model = monkey_v1().to("cuda")

In [ ]:
# next(iter(response_predicting_model.parameters())).device

## Load simulated neuron model

In [ ]:
from laminr.neuron_models.simulated import neuron3_generator as neuron_model_generator

loc = [0.2, .2]
img_res = [100, 100]
neuron_model1, rf_grid = neuron_model_generator(loc, img_res)
xx, yy = rf_grid

loc = [-0.2, -.2]
img_res = [100, 100]
neuron_model2, rf_grid = neuron_model_generator(loc, img_res)
xx, yy = rf_grid

loc = [-0.2, .2]
img_res = [100, 100]
neuron_model3, rf_grid = neuron_model_generator(loc, img_res)
xx, yy = rf_grid

In [ ]:
from laminr.neuron_models.utils.simulated_model import ArbitraryMultiNeuronModel

In [ ]:
response_predicting_model = ArbitraryMultiNeuronModel([neuron_model1, neuron_model2, neuron_model3]).to("cuda")

In [ ]:
response_predicting_model

## Generate MEIs

In [ ]:
import numpy as np
from laminr import get_mei_dict

In [ ]:
# neuron_idxs = [0]
# input_shape = [1, 93, 93]
# required_pixel_min = -1.7876
# required_pixel_max = 2.1919
# std = 0.15
# required_norm = std * np.sqrt(np.product(input_shape[-2:]))

neuron_idxs = [0, 1, 2]
input_shape = [1, 100, 100]
required_pixel_min = -10
required_pixel_max = 10
required_norm = 1

meis_dict = get_mei_dict(
    response_predicting_model,
    neuron_idxs,
    input_shape,
    required_pixel_min,
    required_pixel_max,
    required_norm=required_norm,
)

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
fig, (ax1, ax2, ax3) = plt.subplots(1, 3)

ax1.imshow(meis_dict[0]["mei"][0])
ax2.imshow(meis_dict[1]["mei"][0])
ax3.imshow(meis_dict[2]["mei"][0])

In [ ]:
fig, (ax1, ax2, ax3) = plt.subplots(1, 3)

ax1.imshow(meis_dict[0]["mask"])
ax2.imshow(meis_dict[1]["mask"])
ax3.imshow(meis_dict[2]["mask"])

## Initialize the pipeline

In [ ]:
from laminr import InvarianceManifold

In [ ]:
invariance_manifold = InvarianceManifold(
    response_predicting_model,
    meis_dict,
    required_img_pixel_min=required_pixel_min,
    required_img_pixel_max=required_pixel_max,
    required_img_norm=required_norm,
)

In [ ]:
template_neuron_idx = 0

template = invariance_manifold.learn(
    template_neuron_idx, 
    steps_per_epoch=1,
)

In [ ]:
# template.show()

In [ ]:
target_neuron_idxs = [1, 2]
template = invariance_manifold.match(
    target_neuron_idxs
)

In [ ]:
images_from_learned_template = invariance_manifold.get_images_from_learned_template()
images_from_learned_template.shape

In [ ]:
gif = invariance_manifold.save_learned_template_as_gif()
gif.show()

In [ ]:
images_from_matched_template = invariance_manifold.get_images_from_matched_template(target_neuron_idxs[0])
images_from_matched_template.shape

In [ ]:
gif = invariance_manifold.save_matched_template_as_gif(target_neuron_idxs[1])
gif.show()